In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import datetime
from stocks import StockTrades
import matplotlib.pyplot as plt
from positionslib import read_positions_file


In [ ]:
!"C:\\Program Files\\Git\\usr\\bin\\grep.exe" -v "as of" all.csv > all_filt.csv


In [ ]:
def find_key(key, dct, default=None):
    # Filter keys in the dictionary that are less than or equal to key
    valid_keys = [k for k in dct if k >= key]
    
    if not valid_keys:
        # If no valid keys found, return None 
        return default
    
    # Return the pnl value corresponding to the latest date
    return dct[max(valid_keys)]


In [ ]:
def compute_stock_pnl(symbol, start_dates, end_date, price, qty):
    start_dates=list(sorted(start_dates))

    trades = StockTrades(symbol, start_dates[0], end_date)
    trades.read_data("all_filt.csv")
    # Access the filtered DataFrame
    if trades.df is not None:
        pass    
    else:
        print("No data matching the criteria.")
    
    dct=trades.find_pnl(qty, price)
    pnls=[]
    for dt in start_dates:
        dt1=pd.to_datetime(dt, format='%m/%d/%Y')
        pnls.append(find_key(dt1, dct, default=np.nan))
            # find the latest date in the keys of dct that precedes dt
            # and save the corresponding pnl in out_lst
            # if there is no such date, then store NaN
    return pnls

symbol = "AMD"
# date_format = "%m/%d/%Y"

# # Use datetime.strptime() to parse the date string into a datetime object
# # start_date = datetime.datetime.strptime("01/01/2023", date_format).date()
# # end_date = datetime.datetime.strptime("04/17/2024", date_format).date()

current_price=79.19
start_date = "01/01/2024"
end_date = "12/20/2024"

pos=0
_ =compute_stock_pnl(symbol, [start_date], end_date, current_price, pos)




In [ ]:

start_date = "01/01/2024"
end_date = "12/20/2024"
df=read_positions_file("pos.csv")
df=df.loc[df["Security Type"] != "Option", ["Symbol", "Price", "Quantity"]]
symbols=["GLD", "ARKK", "VNQ", "BIDU", "CBRL", "XBI", "ITB", "FI", "DIA", "CRWD",
          "JACK", "XHB",  "XLK", "XLV",  "VO", "XLY", "CRM", "ADSK", "AMD", "XOP", "HD", "LULU"]
pnls=[]
for symbol in sorted(list(set(symbols))):
    #locate the row with Symbol == symbol; 

    try:
        x=df.loc[df.Symbol == symbol]
        if len(x) ==0:
            p, q=0,0
        else:
            p, q=x.Price.iloc[0], x.Quantity.iloc[0]
        # print(symbol, p, q)
        
        pnl=compute_stock_pnl(symbol, [start_date], end_date, p, q)
        pnls.append([symbol, p, q] + pnl)
    except:
        print("Skipping", symbol)
out_df=pd.DataFrame(pnls, columns=["symbol", "price", "qty", "pnl"])
print("Total pnl", out_df.pnl.sum())
out_df

                                                                            

